In [1]:
%pylab inline
import numpy as np
import time
import sys
import logging
from sklearn.decomposition import TruncatedSVD as tsvd
from h2o4gpu.solvers import TruncatedSVDH2O
!nvidia-smi
import sys
import py3nvml ## pip install -e git+https://github.com/fbcotter/py3nvml#egg=py3nvml
import subprocess
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

Populating the interactive namespace from numpy and matplotlib
Sun Dec  3 07:29:51 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0  On |                  N/A |
| 26%   42C    P5    24W / 250W |   1576MiB / 11172MiB |     12%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:81:00.0 Off |                  N/A |
| 23%   37C    P8    16W / 250W |    242MiB / 11172MiB |      0%      Default |
+------------------------

In [2]:
%%time

#Set up matrix (m x n)
np.random.seed(1234)
m=200000; n=1000
X = np.random.rand(m, n)
k=10 # number of components

CPU times: user 2.03 s, sys: 696 ms, total: 2.73 s
Wall time: 2.73 s


In [3]:
%%time
start_time = time.time()
h2o4gpu_tsvd = TruncatedSVDH2O(n_components=k)
h2o4gpu_tsvd.fit(X)
end_time = time.time() - start_time
printmd("**Total time for h2o4gpu arpack tsvd is " + str(end_time) + "**")
print("h2o4gpu tsvd Singular Values")
print(h2o4gpu_tsvd.singular_values_)
print("h2o4gpu tsvd Components (V^T)")
print(h2o4gpu_tsvd.components_)
print("h2o4gpu tsvd Explained Variance")
print(h2o4gpu_tsvd.explained_variance_)
print("h2o4gpu tsvd Explained Variance Ratio")
print(h2o4gpu_tsvd.explained_variance_ratio_)

**Total time for h2o4gpu arpack tsvd is 6.568532228469849**

h2o4gpu tsvd Singular Values
[ 7072.42822266   138.08740234   138.0827179    137.95184326   137.91082764
   137.89772034   137.77865601   137.74250793   137.65570068   137.64558411]
h2o4gpu tsvd Components (V^T)
[[ 0.03160822  0.03163449  0.03162544 ...,  0.03160141  0.03167794
   0.0315132 ]
 [-0.03652671  0.0144819   0.0087755  ...,  0.05670537 -0.0237855
   0.01040668]
 [ 0.00863585  0.02036315 -0.00762595 ...,  0.01559443  0.02623164
   0.10060279]
 ..., 
 [-0.02592493  0.03186912 -0.02734392 ...,  0.00698451  0.01649839  0.        ]
 [ 0.00631481  0.04533872 -0.01610234 ..., -0.00025566  0.01935858
  -0.06476473]
 [ 0.01102729 -0.07528458 -0.06134396 ..., -0.00144295  0.01815314  0.        ]]
h2o4gpu tsvd Explained Variance
[ 0.08016483  0.08030165  0.08033707  0.08026209  0.08039588  0.08033238
  0.08033585  0.08033044  0.08032776  0.08026112]
h2o4gpu tsvd Explained Variance Ratio
[ 0.00097478  0.00097644  0.00097687  0.00097596  0.00097759  0.00097682
  0.00097686  0.00097679  0

In [4]:
%%time 
start_time = time.time()
model = tsvd(n_components=k, algorithm="arpack")
model.fit(X)
end_time = time.time() - start_time
printmd("**Total time for arpack scikit-learn tsvd is " + str(end_time) + "**")
print("tsvd Singular Values")
print(model.singular_values_)
print("tsvd Components (V^T)")
print(model.components_)
print("tsvd Explained Variance")
print(model.explained_variance_)
print("tsvd Explained Variance Ratio")
print(model.explained_variance_ratio_)

**Total time for arpack scikit-learn tsvd is 41.16011619567871**

tsvd Singular Values
[ 7072.4289909    138.14048601   138.09703705   137.98470993   137.91346777
   137.86160169   137.81041514   137.76470811   137.71770837   137.65846051]
tsvd Components (V^T)
[[ 0.03160822  0.03163449  0.03162544 ...,  0.0316014   0.031678
   0.03151319]
 [ 0.02064035 -0.02891545  0.02041258 ..., -0.03163838 -0.01079971
  -0.05860967]
 [-0.02544365  0.00285468  0.02429651 ...,  0.04693024 -0.0375039
  -0.04562807]
 ..., 
 [-0.06114168  0.02761409 -0.02933754 ...,  0.02465429  0.03333408
   0.01499451]
 [-0.01115856  0.03547422 -0.01811966 ..., -0.00531147 -0.0035131
  -0.09533024]
 [ 0.04268635 -0.04544683 -0.01477452 ..., -0.01384292 -0.01668161
   0.01887234]]
tsvd Explained Variance
[ 0.08306231  0.09541397  0.09535396  0.0951989   0.09510062  0.09502911
  0.09495855  0.09489557  0.09483084  0.09474926]
tsvd Explained Variance Ratio
[ 0.00099672  0.00114494  0.00114422  0.00114236  0.00114118  0.00114032
  0.00113947  0.00113872  0.00113794  0.00113696]
CPU time

In [5]:
%%time
start_time = time.time()
model = tsvd(n_components=k, algorithm="randomized")
model.fit(X)
end_time = time.time() - start_time
printmd("**Total time for randomized scikit-learn tsvd is " + str(end_time) + "**")
print("tsvd Singular Values")
print(model.singular_values_)
print("tsvd Components (V^T)")
print(model.components_)
print("tsvd Explained Variance")
print(model.explained_variance_)
print("tsvd Explained Variance Ratio")
print(model.explained_variance_ratio_)

**Total time for randomized scikit-learn tsvd is 13.020515203475952**

tsvd Singular Values
[ 7072.4289909    133.37031017   133.22144366   133.05844997   132.90668912
   132.83067352   132.75548619   132.67283248   132.49428064   132.39504544]
tsvd Components (V^T)
[[ 0.03160822  0.03163449  0.03162544 ...,  0.0316014   0.031678
   0.03151319]
 [ 0.01463285  0.00081001 -0.0430451  ..., -0.01419612  0.0010795
  -0.0302737 ]
 [-0.03593592 -0.00264458 -0.06579635 ..., -0.03984711  0.02088399
  -0.03138619]
 ..., 
 [-0.01864775 -0.0305044  -0.00625413 ..., -0.03590451 -0.03470779
   0.01215434]
 [-0.02234073  0.03082768  0.00760221 ...,  0.00105271  0.02961436
  -0.02676475]
 [ 0.04186557 -0.0226522   0.00636629 ...,  0.01948014 -0.01271316
   0.02798382]]
tsvd Explained Variance
[ 0.08306231  0.0889382   0.08873977  0.08852276  0.08832094  0.08821994
  0.0881201   0.0880104   0.08777367  0.08764224]
tsvd Explained Variance Ratio
[ 0.00099672  0.00106723  0.00106485  0.00106224  0.00105982  0.00105861
  0.00105741  0.0010561   0.00105326  0.00105168]
CPU tim